In [31]:
import os
import pandas as pd
from geopy.distance import vincenty, geodesic

In [55]:
from tqdm import tqdm

In [2]:
path_to_data = "data"

In [82]:
file = "201809-citibike-tripdata.csv"

data = pd.read_csv(os.path.join(path_to_data, file), parse_dates=['starttime', 'stoptime'])

In [83]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.269,2018-09-01 00:27:20.634,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.281,2018-09-01 00:02:23.481,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.649,2018-09-01 00:55:58.547,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.746,2018-09-01 00:07:38.583,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.315,2018-09-01 02:21:25.308,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


### 1

In [70]:
data.shape.ipynb_checkpoints/

(1877884, 15)

### 2

In [8]:
data["tripduration"].mean() / 60

16.125248648656324

### 3

In [10]:
(data["start station name"] == data["end station name"]).sum()

41364

### 4

In [161]:
data['bikeid'].value_counts()[:1]

33875    559
Name: bikeid, dtype: int64

### 5

In [150]:
data.groupby("bikeid")["tripduration"].mean().sort_values()[-1:]

bikeid
17548    488022.4
Name: tripduration, dtype: float64

### 6

In [18]:
(data["start station id"].isnull()).sum()

716

### 7

In [21]:
data.groupby("usertype")[["tripduration"]].mean() / 60

,tripduration
usertype,
Customer,33.415379
Subscriber,13.332348


### 8

In [121]:
data8 = data[data['start station id'] != data['end station id']].copy()

In [125]:
%%time
data8["distance"] = data8.apply(lambda row: geodesic((row["start station latitude"],
                                                     row["start station longitude"]),
                                                    (row["end station latitude"],
                                                     row["end station longitude"])).km, axis=1)

Wall time: 7min 37s


In [126]:
data8["distance"].mean()

1.8495860872213767

### 9

In [86]:
ind = (data["starttime"].dt.hour >= 18) & (data["starttime"].dt.hour <= 20)

In [158]:
data[ind]["start station id"].value_counts().sort_values(ascending=False).iloc[:1]

519.0    3824
Name: start station id, dtype: int64

### 10

In [98]:
ind = (data["stoptime"].dt.hour >= 6) & (data["stoptime"].dt.hour <= 10)

In [127]:
end_stations = [3140, 3106, 3116, 369]

In [152]:
list(set(end_stations) & set(data[ind]["end station id"]))

[369, 3106, 3140, 3116]